In [21]:

import numpy as np 
from math import pi, atan
def ptGrid(n_in,l_in):
    nList = []
    for i in range(n_in[0]):
        for j in range(n_in[1]):
            for k in range(n_in[2]):
                nList.append([round(i*l_in[0],10),round(j*l_in[1],10),round(k*l_in[2],10)])
    return nList

def calAngle(pt0,pt1):
    xyL  = ((pt1[1]-pt0[1])**2 + (pt1[0]-pt0[0])**2)**0.5

    if pt1[2]-pt0[2] == 0:
        phi = 0
    elif xyL == 0:
        phi = pi/2
    else:
        phi   = atan((pt1[2]-pt0[2])/xyL)
    
    if pt1[1]-pt0[1] == 0:
        theta = pi/2
    elif pt1[0]-pt0[0] == 0:
        theta = 0
    else:
        theta = atan((pt1[1]-pt0[1])/(pt1[0]-pt0[0]))
    
    return [phi,theta]

def genLine(pt, a_i, E_i):
    checkList = []
    crv = []
    for i, pt_i in enumerate(pt):
        for j,pt_j in enumerate(pt[i+1:]):
            [phi,theta] = calAngle(pt_j,pt_i)
            phi_ = round(phi,10)
            theta_ = round(theta,10)
            
            if [pt_i, phi_, theta_] not in checkList:
                checkList.append([pt_i, phi_, theta_])
                crv.append([i,i+j+1, a_i, E_i])
    return crv

def genBC(cPt,fPt,nList):
    cId = [nList.index(cPti['n']) for cPti in cPt]
    fId = [nList.index(fPti['n']) for fPti in fPt]
    cList = []
    fList = []

    for i, cIdi in enumerate(cId):
        cList.append([cIdi] + [cPt[i]['dof'][0]] + [cPt[i]['dof'][1]] + [cPt[i]['dof'][2]] + [cPt[i]['mag']])

    for i, fIdi in enumerate(fId):
        fList.append([fIdi] + [fPt[i]['dof'][0]] + [fPt[i]['dof'][1]] + [fPt[i]['dof'][2]] + [fPt[i]['mag']])
    return [cList,fList]

import os

def f(l):
    return [item for sublist in l for item in sublist]

def writeList(wDir, mName, m, l,fName):
    
    fPath = wDir+'/'+str(m)+mName
    # Check whether the specified path exists or not
    if not os.path.exists(fPath):
      # Create a new directory because it does not exist 
      os.makedirs(fPath)
      
    np.savetxt(wDir+'/'+str(m)+mName+'/'+fName+'.csv', l, delimiter=',')
           


In [25]:

l_total = 0.15 # m
n_node = 5
s = l_total/(n_node-1)
n_in = [n_node]*3
l_in = [s]*3

maxA = (s/3)**2*np.pi
print(maxA)

a_i = 0.01*0.01*np.pi
E_i = 820e6

nList = ptGrid(n_in,l_in)
mList = genLine(nList, a_i, E_i)

# constraint nodes
cPt =[{'n':[      0,      0,      0],'dof':[1,1,1],'mag':0},
      {'n':[      0,l_total,      0],'dof':[1,1,1],'mag':0},
      {'n':[      0,      0,l_total],'dof':[1,1,1],'mag':0},
      {'n':[      0,l_total,l_total],'dof':[1,1,1],'mag':0}]

# force nodes
if n_node % 2 ==0:
    fPt = [{'n':[l_total, round((l_total-s)/2.,10), round((l_total-s)/2.,10)],'dof':[0,1,0],'mag':-25},
           {'n':[l_total, round((l_total-s)/2.,10), round((l_total+s)/2.,10)],'dof':[0,1,0],'mag':-25},
           {'n':[l_total, round((l_total+s)/2.,10), round((l_total-s)/2.,10)],'dof':[0,1,0],'mag':-25},
           {'n':[l_total, round((l_total+s)/2.,10), round((l_total+s)/2.,10)],'dof':[0,1,0],'mag':-25}]
else:
    fPt = [{'n':[l_total, l_total/2., l_total/2.],'dof':[0,1,0],'mag':-25}]

[cList,fList] = genBC(cPt,fPt,nList)

wDir = '/Users/tianchen/Research/trussOpt/input'
mName = '_bar'

print("num of nodes: "       + str(len(nList)))
print("num of members: "     + str(len(mList)))
print("num of constraints: " + str(len(cList)))
print("num of forces: "      + str(len(fList)))

mNum = len(mList)
wTog = True
if wTog == True:
    writeList(wDir, mName, mNum, nList, 'nodes')
    writeList(wDir, mName, mNum, mList, 'members')
    writeList(wDir, mName, mNum, cList, 'constraints')
    writeList(wDir, mName, mNum, fList, 'forces')

0.0004908738521234051
num of nodes: 125
num of members: 6412
num of constraints: 4
num of forces: 1
